In [1]:
!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo

import re
import os
import sys
import json

import logging
import numpy as np
import pandas as pd

import logging
logging.getLogger('tensorflow').disabled = True

import tensorflow_hub as hub

import tensorflow.keras
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

from sklearn.model_selection import train_test_split

if not 'bert_repo' in sys.path:
    sys.path.insert(0, 'bert_repo')

from modeling import BertModel, BertConfig
from tokenization import FullTokenizer, convert_to_unicode
from extract_features import InputExample, convert_examples_to_features


#get TF logger 
log = logging.getLogger('tensorflow')
log.handlers = []

C:\Users\sayee\anaconda3\envs\yo\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\sayee\anaconda3\envs\yo\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\Users\sayee\anaconda3\envs\yo\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\sayee\anaconda3\envs\yo\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# Base Imports 
import json
import pandas as pd
from pprint import pprint
import argparse
import io
import re
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from os import listdir
from os.path import isfile, join
# Import our dependencies
import tensorflow as tf
#tf.disable_v2_behavior()
import pandas as pd
import tensorflow_hub as hub
from keras import backend as K
#from keras_contrib.layers import CRF


import tensorflow.keras.layers as layers
from tensorflow.keras.models import Model, load_model,Sequential
from tensorflow.keras.layers import Layer, InputSpec,Lambda
import numpy as np
from sklearn.model_selection import train_test_split
import webbrowser
batch_size = 1

#embed = hub.Module("https://tfhub.dev/google/elmo/2", trainable=False)
# Initialize session and make sure tensorflow doesn't hog all of the gpu for itself by setting the allow growth config
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)
K.set_session(sess)

Using TensorFlow backend.


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam as Adam
from sklearn.utils import class_weight

In [4]:
def build_module_fn(config_path, vocab_path, do_lower_case=True):

    def bert_module_fn(is_training):
        """Spec function for a token embedding module."""

        input_ids = tensorflow.placeholder(shape=[None, None], dtype= tensorflow.int32, name="input_ids")
        input_mask =  tensorflow.placeholder(shape=[None, None], dtype= tensorflow.int32, name="input_mask")
        token_type =  tensorflow.placeholder(shape=[None, None], dtype= tensorflow.int32, name="segment_ids")

        config = BertConfig.from_json_file(config_path)
        model = BertModel(config=config, is_training=is_training,
                          input_ids=input_ids, input_mask=input_mask, token_type_ids=token_type)
          
        seq_output = model.all_encoder_layers[-1]
        pool_output = model.get_pooled_output()

        config_file =  tensorflow.constant(value=config_path, dtype= tensorflow.string, name="config_file")
        vocab_file =  tensorflow.constant(value=vocab_path, dtype= tensorflow.string, name="vocab_file")
        lower_case =  tensorflow.constant(do_lower_case)

        tensorflow.add_to_collection( tensorflow.GraphKeys.ASSET_FILEPATHS, config_file)
        tensorflow.add_to_collection( tensorflow.GraphKeys.ASSET_FILEPATHS, vocab_file)
        
        input_map = {"input_ids": input_ids,
                     "input_mask": input_mask,
                     "segment_ids": token_type}
        
        output_map = {"pooled_output": pool_output,
                      "sequence_output": seq_output}

        output_info_map = {"vocab_file": vocab_file,
                           "do_lower_case": lower_case}
                
        hub.add_signature(name="tokens", inputs=input_map, outputs=output_map)
        hub.add_signature(name="tokenization_info", inputs={}, outputs=output_info_map)

    return bert_module_fn

In [5]:
def read_examples(str_list):
    #"""Read a list of `InputExample`s from a list of strings."""
    unique_id = 0
    for s in str_list:
        line = convert_to_unicode(s)
        if not line:
            continue
        line = line.strip()
        text_a = None
        text_b = None
        m = re.match(r"^(.*) \|\|\| (.*)$", line)
        if m is None:
            text_a = line
        else:
            text_a = m.group(1)
            text_b = m.group(2)
        yield InputExample(unique_id=unique_id, text_a=text_a, text_b=text_b)
        unique_id += 1

In [6]:
def features_to_arrays(features):

    all_input_ids = []
    all_input_mask = []
    all_segment_ids = []

    for feature in features:
        all_input_ids.append(feature.input_ids)
        all_input_mask.append(feature.input_mask)
        all_segment_ids.append(feature.input_type_ids)

    return (np.array(all_input_ids, dtype='int32'), 
            np.array(all_input_mask, dtype='int32'), 
            np.array(all_segment_ids, dtype='int32'))

In [7]:
def build_preprocessor(voc_path, seq_len, lower=True):
    tokenizer = FullTokenizer(vocab_file=voc_path, do_lower_case=lower)
  
    def strings_to_arrays(sents):

      sents = np.atleast_1d(sents).reshape((-1,))

      examples = []
      for example in read_examples(sents):
          examples.append(example)

      features = convert_examples_to_features(examples, seq_len, tokenizer)
      arrays = features_to_arrays(features)
      return arrays
  
    return strings_to_arrays

In [8]:
class BertLayer(tensorflow.keras.layers.Layer):
    def __init__(self, bert_path, seq_len=64, n_tune_layers=3, 
                 pooling="cls", do_preprocessing=True, verbose=False,
                 tune_embeddings=False, trainable=True, **kwargs):

        self.trainable = trainable
        self.n_tune_layers = n_tune_layers
        self.tune_embeddings = tune_embeddings
        self.do_preprocessing = do_preprocessing

        self.verbose = verbose
        self.seq_len = seq_len
        self.pooling = pooling
        self.bert_path = bert_path

        self.var_per_encoder = 16
        if self.pooling not in ["cls", "mean", None]:
            raise NameError(
                f"Undefined pooling type (must be either 'cls', 'mean', or None, but is {self.pooling}"
            )

        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):

        self.bert = hub.Module(self.build_abspath(self.bert_path), 
                               trainable=self.trainable, name=f"{self.name}_module")

        trainable_layers = []
        if self.tune_embeddings:
            trainable_layers.append("embeddings")

        if self.pooling == "cls":
            trainable_layers.append("pooler")

        if self.n_tune_layers > 0:
            encoder_var_names = [var.name for var in self.bert.variables if 'encoder' in var.name]
            n_encoder_layers = int(len(encoder_var_names) / self.var_per_encoder)
            for i in range(self.n_tune_layers):
                trainable_layers.append(f"encoder/layer_{str(n_encoder_layers - 1 - i)}/")
        
        # Add module variables to layer's trainable weights
        for var in self.bert.variables:
            if any([l in var.name for l in trainable_layers]):
                self._trainable_weights.append(var)
            else:
                self._non_trainable_weights.append(var)

#         if self.verbose:
#             print("*** TRAINABLE VARS *** ")
#             for var in self._trainable_weights:
#                 print(var)

        self.build_preprocessor()
        self.initialize_module()

        super(BertLayer, self).build(input_shape)

    def build_abspath(self, path):
        if path.startswith("https://") or path.startswith("gs://"):
          return path
        else:
          return os.path.abspath(path)

    def build_preprocessor(self):
        sess = tensorflow.keras.backend.get_session()
        tokenization_info = self.bert(signature="tokenization_info", as_dict=True)
        vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                              tokenization_info["do_lower_case"]])
        self.preprocessor = build_preprocessor(vocab_file, self.seq_len, do_lower_case)

    def initialize_module(self):
        sess = tensorflow.keras.backend.get_session()
        
        vars_initialized = sess.run([tensorflow.is_variable_initialized(var) 
                                     for var in self.bert.variables])

        uninitialized = []
        for var, is_initialized in zip(self.bert.variables, vars_initialized):
            if not is_initialized:
                uninitialized.append(var)

        if len(uninitialized):
            sess.run(tensorflow.variables_initializer(uninitialized))

    def call(self, input):

        if self.do_preprocessing:
          input = tensorflow.numpy_function(self.preprocessor, 
                                    [input], [tensorflow.int32, tensorflow.int32, tensorflow.int32,], 
                                    name='preprocessor')
          for feature in input:
            feature.set_shape((None, self.seq_len))
        
        input_ids, input_mask, segment_ids = input
        
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        output = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)
        
        if self.pooling == "cls":
            pooled = output["pooled_output"]
        else:
            result = output["sequence_output"]
            
            input_mask = tf.cast(input_mask, tf.float32)
            mul_mask = lambda x, m: x * tf.expand_dims(m, axis=-1)
            masked_reduce_mean = lambda x, m: tf.reduce_sum(mul_mask(x, m), axis=1) / (
                    tf.reduce_sum(m, axis=1, keepdims=True) + 1e-10)
            
            if self.pooling == "mean":
              pooled = masked_reduce_mean(result, input_mask)
            else:
              pooled = mul_mask(result, input_mask)

        return pooled

    def get_config(self):
        config_dict = {
            "bert_path": self.bert_path, 
            "seq_len": self.seq_len,
            "pooling": self.pooling,
            "n_tune_layers": self.n_tune_layers,
            "tune_embeddings": self.tune_embeddings,
            "do_preprocessing": self.do_preprocessing,
            "verbose": self.verbose
        }
        super(BertLayer, self).get_config()
        return config_dict

In [9]:
df = pd.read_csv('C:\\Users\\sayee\\Python-ML(Udemy)\\NLP Research Paper\\Invoice_data_extraction\\labelled dataset\\HybridModel2.csv')

In [10]:
df = df.drop('Unnamed: 0',axis=1)

In [11]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(df['label']),
                                                 df['label'])

C:\Users\sayee\anaconda3\envs\yo\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass classes=['GST' 'address' 'date' 'description' 'email' 'name' 'none' 'ph_no' 'sac'
 'tax_percent' 'tot_in_words' 'value'], y=0            GST
1           none
2        address
3           none
4           none
          ...   
43617        GST
43618       none
43619       none
43620       none
43621       none
Name: label, Length: 43622, dtype: object as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [14]:
df['label'].value_counts()

none            25164
value            6594
date             2929
address          2363
name             1766
tax_percent      1185
description       911
GST               772
sac               680
tot_in_words      637
ph_no             388
email             233
Name: label, dtype: int64

In [15]:
np.unique(df['label'])

array(['GST', 'address', 'date', 'description', 'email', 'name', 'none',
       'ph_no', 'sac', 'tax_percent', 'tot_in_words', 'value'],
      dtype=object)

In [16]:
df.columns

Index(['Blocks', 'Source', 'PageWidth', 'PageHeight', 'TopLeftX', 'TopLeftY',
       'TopRightX', 'TopRightY', 'BottomRightX', 'BottomRightY', 'BottomLeftX',
       'BottomLeftY', 'label', 'verification', 'Confidence', 'MP', 'MidPointX',
       'MidPointY', 'ClosestB1', 'ClosestB2', 'ClosestB3', 'ClosestB4',
       'ClosestBC1X', 'ClosestBC1Y', 'ClosestBC2X', 'ClosestBC2Y',
       'ClosestBC3X', 'ClosestBC3Y', 'ClosestBC4X', 'ClosestBC4Y', 'PredLabel',
       'ClosestB1Label', 'ClosestB2Label', 'ClosestB3Label', 'ClosestB4Label'],
      dtype='object')

In [12]:
for index,rows in df.iterrows():
    df['ClosestB1Label'][index]='ClosestB1Label_'+ df['ClosestB1Label'][index]
    df['ClosestB2Label'][index]='ClosestB2Label_'+ df['ClosestB2Label'][index]
    df['ClosestB3Label'][index]='ClosestB3Label_'+ df['ClosestB3Label'][index]
    df['ClosestB4Label'][index]='ClosestB4Label_'+ df['ClosestB4Label'][index]

C:\Users\sayee\anaconda3\envs\yo\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\sayee\anaconda3\envs\yo\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\sayee\anaconda3\envs\yo\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

In [18]:
df

,Blocks,Source,PageWidth,PageHeight,TopLeftX,TopLeftY,TopRightX,TopRightY,BottomRightX,BottomRightY,...,ClosestBC2Y,ClosestBC3X,ClosestBC3Y,ClosestBC4X,ClosestBC4Y,PredLabel,ClosestB1Label,ClosestB2Label,ClosestB3Label,ClosestB4Label
0,GST NO.37AADCV9696D2ZQ,219143549.pdf,596,842,0.780201,0.020190,0.979866,0.020190,0.979866,0.032067,...,0.2090,0.8725,0.2482,0.5159,0.0974,GST,ClosestB1Label_none,ClosestB2Label_none,ClosestB3Label_date,ClosestB4Label_address
1,t ' ul Mag Gemoval,219143549.pdf,596,842,0.664430,0.024941,0.843960,0.000000,0.850671,0.019002,...,0.0974,0.8297,0.2090,0.5747,0.2084,none,ClosestB1Label_GST,ClosestB2Label_address,ClosestB3Label_none,ClosestB4Label_none
2,"VASHANTH OUTDOOR MEDIA PVT.LTD.· H.0: # 517,In...",219143549.pdf,596,842,0.151007,0.051069,0.880873,0.054632,0.880873,0.140143,...,0.2072,0.6267,0.2500,0.7542,0.0095,address,ClosestB1Label_none,ClosestB2Label_none,ClosestB3Label_none,ClosestB4Label_none
3,State Code: 37,219143549.pdf,596,842,0.340604,0.200713,0.442953,0.200713,0.442953,0.213777,...,0.2458,0.2156,0.2132,0.2156,0.2132,none,ClosestB1Label_address,ClosestB2Label_name,ClosestB3Label_none,ClosestB4Label_address
4,TAX INVOICE,219143549.pdf,596,842,0.526846,0.201900,0.622483,0.203088,0.622483,0.213777,...,0.3058,0.5159,0.0974,0.3918,0.2072,none,ClosestB1Label_none,ClosestB2Label_none,ClosestB3Label_address,ClosestB4Label_none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43617,GST ID: 33AAGCT1726K1ZC,220310188(sys india).pdf,596,842,0.132550,0.899050,0.340604,0.900238,0.340604,0.909739,...,0.8201,0.0579,0.9038,0.0394,0.9614,GST,ClosestB1Label_none,ClosestB2Label_tot_in_words,ClosestB3Label_none,ClosestB4Label_none
43618,ISE SEE REVERSE FOR RTGS AND OTHER DETAILS ),220310188(sys india).pdf,596,842,0.000000,0.920428,0.280201,0.921615,0.280201,0.934679,...,0.9050,0.0394,0.9614,0.0721,0.7833,none,ClosestB1Label_none,ClosestB2Label_GST,ClosestB3Label_none,ClosestB4Label_none
43619,is Summ,220310188(sys india).pdf,596,842,0.864094,0.914489,1.005034,0.926366,1.000000,0.956057,...,0.8219,0.8238,0.7435,0.9664,0.7162,none,ClosestB1Label_none,ClosestB2Label_value,ClosestB3Label_tax_percent,ClosestB4Label_value
43620,- Tear Sheets O.E,220310188(sys india).pdf,596,842,0.001678,0.950119,0.077181,0.950119,0.077181,0.972684,...,0.9281,0.2366,0.9050,0.0721,0.7833,none,ClosestB1Label_none,ClosestB2Label_none,ClosestB3Label_GST,ClosestB4Label_none


In [13]:
label_dummies1 = pd.get_dummies(df.ClosestB1Label)
df = pd.concat([df, label_dummies1], axis=1)

label_dummies2 = pd.get_dummies(df.ClosestB2Label)
df = pd.concat([df, label_dummies2], axis=1)

label_dummies3 = pd.get_dummies(df.ClosestB3Label)
df = pd.concat([df, label_dummies3], axis=1)

label_dummies4 = pd.get_dummies(df.ClosestB4Label)
df = pd.concat([df, label_dummies4], axis=1)

In [18]:
df.columns

Index(['Blocks', 'Source', 'PageWidth', 'PageHeight', 'TopLeftX', 'TopLeftY',
       'TopRightX', 'TopRightY', 'BottomRightX', 'BottomRightY', 'BottomLeftX',
       'BottomLeftY', 'label', 'verification', 'Confidence', 'MP', 'MidPointX',
       'MidPointY', 'ClosestB1', 'ClosestB2', 'ClosestB3', 'ClosestB4',
       'ClosestBC1X', 'ClosestBC1Y', 'ClosestBC2X', 'ClosestBC2Y',
       'ClosestBC3X', 'ClosestBC3Y', 'ClosestBC4X', 'ClosestBC4Y', 'PredLabel',
       'ClosestB1Label', 'ClosestB2Label', 'ClosestB3Label', 'ClosestB4Label',
       'ClosestB1Label_GST', 'ClosestB1Label_address', 'ClosestB1Label_date',
       'ClosestB1Label_description', 'ClosestB1Label_email',
       'ClosestB1Label_name', 'ClosestB1Label_none', 'ClosestB1Label_ph_no',
       'ClosestB1Label_sac', 'ClosestB1Label_tax_percent',
       'ClosestB1Label_tot_in_words', 'ClosestB1Label_value',
       'ClosestB2Label_GST', 'ClosestB2Label_address', 'ClosestB2Label_date',
       'ClosestB2Label_description', 'ClosestB2La

In [19]:
df

,Blocks,Source,PageWidth,PageHeight,TopLeftX,TopLeftY,TopRightX,TopRightY,BottomRightX,BottomRightY,...,ClosestB4Label_date,ClosestB4Label_description,ClosestB4Label_email,ClosestB4Label_name,ClosestB4Label_none,ClosestB4Label_ph_no,ClosestB4Label_sac,ClosestB4Label_tax_percent,ClosestB4Label_tot_in_words,ClosestB4Label_value
0,GST NO.37AADCV9696D2ZQ,219143549.pdf,596,842,0.780201,0.020190,0.979866,0.020190,0.979866,0.032067,...,0,0,0,0,0,0,0,0,0,0
1,t ' ul Mag Gemoval,219143549.pdf,596,842,0.664430,0.024941,0.843960,0.000000,0.850671,0.019002,...,0,0,0,0,1,0,0,0,0,0
2,"VASHANTH OUTDOOR MEDIA PVT.LTD.· H.0: # 517,In...",219143549.pdf,596,842,0.151007,0.051069,0.880873,0.054632,0.880873,0.140143,...,0,0,0,0,1,0,0,0,0,0
3,State Code: 37,219143549.pdf,596,842,0.340604,0.200713,0.442953,0.200713,0.442953,0.213777,...,0,0,0,0,0,0,0,0,0,0
4,TAX INVOICE,219143549.pdf,596,842,0.526846,0.201900,0.622483,0.203088,0.622483,0.213777,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43617,GST ID: 33AAGCT1726K1ZC,220310188(sys india).pdf,596,842,0.132550,0.899050,0.340604,0.900238,0.340604,0.909739,...,0,0,0,0,1,0,0,0,0,0
43618,ISE SEE REVERSE FOR RTGS AND OTHER DETAILS ),220310188(sys india).pdf,596,842,0.000000,0.920428,0.280201,0.921615,0.280201,0.934679,...,0,0,0,0,1,0,0,0,0,0
43619,is Summ,220310188(sys india).pdf,596,842,0.864094,0.914489,1.005034,0.926366,1.000000,0.956057,...,0,0,0,0,0,0,0,0,0,1
43620,- Tear Sheets O.E,220310188(sys india).pdf,596,842,0.001678,0.950119,0.077181,0.950119,0.077181,0.972684,...,0,0,0,0,1,0,0,0,0,0


In [14]:
df['Blocks']=df['Blocks'].str.lower()
traindf=pd.get_dummies(df,columns=['label'])
x = traindf[["Blocks",'ClosestB1', 'ClosestB2', 'ClosestB3', 'ClosestB4','ClosestB1Label_GST', 'ClosestB1Label_address', 'ClosestB1Label_date','ClosestB1Label_description', 'ClosestB1Label_email',
       'ClosestB1Label_name', 'ClosestB1Label_none', 'ClosestB1Label_ph_no',
       'ClosestB1Label_sac', 'ClosestB1Label_tax_percent',
       'ClosestB1Label_tot_in_words', 'ClosestB1Label_value',
       'ClosestB2Label_GST', 'ClosestB2Label_address', 'ClosestB2Label_date',
       'ClosestB2Label_description', 'ClosestB2Label_email',
       'ClosestB2Label_name', 'ClosestB2Label_none', 'ClosestB2Label_ph_no',
       'ClosestB2Label_sac', 'ClosestB2Label_tax_percent',
       'ClosestB2Label_tot_in_words', 'ClosestB2Label_value',
       'ClosestB3Label_GST', 'ClosestB3Label_address', 'ClosestB3Label_date',
       'ClosestB3Label_description', 'ClosestB3Label_email',
       'ClosestB3Label_name', 'ClosestB3Label_none', 'ClosestB3Label_ph_no',
       'ClosestB3Label_sac', 'ClosestB3Label_tax_percent',
       'ClosestB3Label_tot_in_words', 'ClosestB3Label_value',
       'ClosestB4Label_GST', 'ClosestB4Label_address', 'ClosestB4Label_date',
       'ClosestB4Label_description', 'ClosestB4Label_email',
       'ClosestB4Label_name', 'ClosestB4Label_none', 'ClosestB4Label_ph_no',
       'ClosestB4Label_sac', 'ClosestB4Label_tax_percent',
       'ClosestB4Label_tot_in_words', 'ClosestB4Label_value', 'MidPointX', 'MidPointY', 'ClosestBC1X', 'ClosestBC1Y', 'ClosestBC2X', 'ClosestBC2Y',
       'ClosestBC3X', 'ClosestBC3Y', 'ClosestBC4X', 'ClosestBC4Y'
       ]]
unique_columns=np.unique('label_'+ df['label'])
y= traindf[unique_columns]
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.15, random_state=420)

In [21]:
y_test

,label_GST,label_address,label_date,label_description,label_email,label_name,label_none,label_ph_no,label_sac,label_tax_percent,label_tot_in_words,label_value
11737,0,0,0,0,0,0,1,0,0,0,0,0
42564,0,0,0,0,0,0,1,0,0,0,0,0
2259,0,0,0,0,0,0,1,0,0,0,0,0
15492,0,0,0,0,0,0,1,0,0,0,0,0
5212,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
28210,0,0,0,0,0,0,1,0,0,0,0,0
9494,0,0,0,0,0,0,1,0,0,0,0,0
15206,0,0,0,0,0,0,0,0,0,0,0,1
2342,0,0,0,0,0,0,0,0,0,0,0,1


In [22]:
df.columns

Index(['Blocks', 'Source', 'PageWidth', 'PageHeight', 'TopLeftX', 'TopLeftY',
       'TopRightX', 'TopRightY', 'BottomRightX', 'BottomRightY', 'BottomLeftX',
       'BottomLeftY', 'label', 'verification', 'Confidence', 'MP', 'MidPointX',
       'MidPointY', 'ClosestB1', 'ClosestB2', 'ClosestB3', 'ClosestB4',
       'ClosestBC1X', 'ClosestBC1Y', 'ClosestBC2X', 'ClosestBC2Y',
       'ClosestBC3X', 'ClosestBC3Y', 'ClosestBC4X', 'ClosestBC4Y', 'PredLabel',
       'ClosestB1Label', 'ClosestB2Label', 'ClosestB3Label', 'ClosestB4Label',
       'ClosestB1Label_GST', 'ClosestB1Label_address', 'ClosestB1Label_date',
       'ClosestB1Label_description', 'ClosestB1Label_email',
       'ClosestB1Label_name', 'ClosestB1Label_none', 'ClosestB1Label_ph_no',
       'ClosestB1Label_sac', 'ClosestB1Label_tax_percent',
       'ClosestB1Label_tot_in_words', 'ClosestB1Label_value',
       'ClosestB2Label_GST', 'ClosestB2Label_address', 'ClosestB2Label_date',
       'ClosestB2Label_description', 'ClosestB2La

In [15]:
def get_model3():
    model3 = Sequential()
    INP1 = layers.Input(shape=(1,), dtype="string", name="sentence1")
    INP2 = layers.Input(shape=(1,), dtype="string", name="sentence2")
    INP3 = layers.Input(shape=(1,), dtype="string", name="sentence3")
    INP4 = layers.Input(shape=(1,), dtype="string", name="sentence4")
    INP5 = layers.Input(shape=(1,), dtype="string", name="sentence5")
    inp1 = layers.Input(shape=(1,))
    inp2 = layers.Input(shape=(1,))
    inp3 = layers.Input(shape=(1,))
    inp4 = layers.Input(shape=(1,))
    inp5 = layers.Input(shape=(1,))
    inp6 = layers.Input(shape=(1,))
    inp7 = layers.Input(shape=(1,))
    inp8 = layers.Input(shape=(1,))
    inp9 = layers.Input(shape=(1,))
    inp10 = layers.Input(shape=(1,))
    inp11 = layers.Input(shape=(1,))
    inp12 = layers.Input(shape=(1,))
    inp13 = layers.Input(shape=(1,))
    inp14 = layers.Input(shape=(1,))
    inp15 = layers.Input(shape=(1,))
    inp16 = layers.Input(shape=(1,))
    inp17 = layers.Input(shape=(1,))
    inp18 = layers.Input(shape=(1,))
    inp19 = layers.Input(shape=(1,))
    inp20 = layers.Input(shape=(1,))
    inp21 = layers.Input(shape=(1,))
    inp22 = layers.Input(shape=(1,))
    inp23 = layers.Input(shape=(1,))
    inp24 = layers.Input(shape=(1,))
    inp25 = layers.Input(shape=(1,))
    inp26 = layers.Input(shape=(1,))
    inp27= layers.Input(shape=(1,))
    inp28 = layers.Input(shape=(1,))
    inp29 = layers.Input(shape=(1,))
    inp30 = layers.Input(shape=(1,))
    inp31 = layers.Input(shape=(1,))
    inp32 = layers.Input(shape=(1,))
    inp33 = layers.Input(shape=(1,))
    inp34 = layers.Input(shape=(1,))
    inp35 = layers.Input(shape=(1,))
    inp36 = layers.Input(shape=(1,))
    inp37 = layers.Input(shape=(1,))
    inp38 = layers.Input(shape=(1,))
    inp39 = layers.Input(shape=(1,))
    inp40 = layers.Input(shape=(1,))
    inp41 = layers.Input(shape=(1,))
    inp42 = layers.Input(shape=(1,))
    inp43 = layers.Input(shape=(1,))
    inp44 = layers.Input(shape=(1,))
    inp45 = layers.Input(shape=(1,))
    inp46 = layers.Input(shape=(1,))
    inp47 = layers.Input(shape=(1,))
    inp48 = layers.Input(shape=(1,))
    
    def emb_layer(inp):
        x = BertLayer(bert_path="./bert-module/", seq_len=48, tune_embeddings=False,
                        pooling='cls', n_tune_layers=1, verbose=False)(inp)
        return x
    X = BertLayer(bert_path="./bert-module/", seq_len=48, tune_embeddings=False,
                        pooling='cls', n_tune_layers=2, verbose=False)(INP1)
    
    x = layers.concatenate([
                    emb_layer(INP2),
                    emb_layer(INP3),
                    emb_layer(INP4),
                    emb_layer(INP5),
                    ])
    
    x = layers.concatenate([X,x])
    
    ClosestLabels = layers.concatenate([x,inp1,inp2,inp3,inp4,inp5,inp6,inp7,inp8,inp9,inp10,inp11,inp12,inp13,inp14,inp15,inp16,inp17,inp18,inp19,inp20,inp21,inp22,inp23,inp24,inp25,inp26,inp27,inp28,inp29,inp30,inp31,inp32,inp33,inp34,inp35,inp36,inp37,inp38,inp39,inp40,inp41,inp42,inp43,inp44,inp45,inp46,inp47,inp48])
    
    mpx =   layers.Input(shape=(1,),   name="MidpointX")
    mpy =   layers.Input(shape=(1,),   name="MidpointY")
    
    
    y = layers.concatenate([ClosestLabels,mpx,mpy]) 
    

    
    CB1X = layers.Input(shape=(1,),   name="CB1MidpointX")
    CB1Y = layers.Input(shape=(1,),   name="CB1MidpointY")
    CB2X = layers.Input(shape=(1,),   name="CB2MidpointX")
    CB2Y = layers.Input(shape=(1,),   name="CB2MidpointY")
    CB3X = layers.Input(shape=(1,),   name="CB3MidpointX")
    CB3Y = layers.Input(shape=(1,),   name="CB3MidpointY")
    CB4X = layers.Input(shape=(1,),   name="CB4MidpointX")
    CB4Y = layers.Input(shape=(1,),   name="CB4MidpointY")
    
    Y = layers.concatenate([y,CB1X,CB1Y,CB2X,CB2Y,CB3X,CB3Y,CB4X,CB4Y])    
    
    dense1 = layers.Dense(512, activation='relu')(Y)
    dense2 = layers.Dense(256, activation='relu')(dense1)
    pred = layers.Dense(12, activation='softmax')(dense2)

    model3 = Model(inputs=[INP1,INP2,INP3,INP4,INP5,inp1,inp2,inp3,inp4,inp5,inp6,inp7,inp8,inp9,inp10,inp11,inp12,inp13,inp14,inp15,inp16,inp17,inp18,inp19,inp20,inp21,inp22,inp23,inp24,inp25,inp26,inp27,inp28,inp29,inp30,inp31,inp32,inp33,inp34,inp35,inp36,inp37,inp38,inp39,inp40,inp41,inp42,inp43,inp44,inp45,inp46,inp47,inp48,mpx,mpy,CB1X,CB1Y,CB2X,CB2Y,CB3X,CB3Y,CB4X,CB4Y], outputs=pred)
    model3.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=0.00005, ),
                  metrics=['accuracy']
                 )

    return model3
model3 = get_model3()
model3.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sentence2 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
sentence3 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
sentence4 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
sentence5 (InputLayer)          [(None, 1)]          0                                            
______________________________________________________________________________________________

In [ ]:
model3.fit([x_train['Blocks'], x_train['ClosestB1'], x_train['ClosestB2'], x_train['ClosestB3'], x_train['ClosestB4'], 
            x_train['ClosestB1Label_GST'], x_train['ClosestB1Label_address'], x_train['ClosestB1Label_date'],
       x_train['ClosestB1Label_description'], x_train['ClosestB1Label_email'],
       x_train['ClosestB1Label_name'], x_train['ClosestB1Label_none'], x_train['ClosestB1Label_ph_no'],
       x_train['ClosestB1Label_sac'], x_train['ClosestB1Label_tax_percent'],
       x_train['ClosestB1Label_tot_in_words'], x_train['ClosestB1Label_value'],
            
       x_train['ClosestB2Label_GST'], x_train['ClosestB2Label_address'], x_train['ClosestB2Label_date'],
       x_train['ClosestB2Label_description'], x_train['ClosestB2Label_email'],
       x_train['ClosestB2Label_name'], x_train['ClosestB2Label_none'], x_train['ClosestB2Label_ph_no'],
       x_train['ClosestB2Label_sac'], x_train['ClosestB2Label_tax_percent'],
       x_train['ClosestB2Label_tot_in_words'], x_train['ClosestB2Label_value'],    
            
       x_train['ClosestB3Label_GST'], x_train['ClosestB3Label_address'], x_train['ClosestB3Label_date'],
       x_train['ClosestB3Label_description'], x_train['ClosestB3Label_email'],
       x_train['ClosestB3Label_name'], x_train['ClosestB3Label_none'], x_train['ClosestB3Label_ph_no'],
       x_train['ClosestB3Label_sac'], x_train['ClosestB3Label_tax_percent'],
       x_train['ClosestB3Label_tot_in_words'], x_train['ClosestB3Label_value'],
            
       x_train['ClosestB4Label_GST'], x_train['ClosestB4Label_address'], x_train['ClosestB4Label_date'],
       x_train['ClosestB4Label_description'], x_train['ClosestB4Label_email'],
       x_train['ClosestB4Label_name'], x_train['ClosestB4Label_none'], x_train['ClosestB4Label_ph_no'],
       x_train['ClosestB4Label_sac'], x_train['ClosestB4Label_tax_percent'],
       x_train['ClosestB4Label_tot_in_words'], x_train['ClosestB4Label_value'], 
            
            
            
            
            
       x_train['MidPointX'],
       x_train['MidPointY'], 
       
            
       x_train['ClosestBC1X'],  x_train['ClosestBC1Y'],  x_train['ClosestBC2X'], x_train['ClosestBC2Y'], x_train['ClosestBC3X'], x_train['ClosestBC3Y'], x_train['ClosestBC4X'], x_train['ClosestBC4Y']],
      
       y_train,
         
        validation_data=([x_test['Blocks'], x_test['ClosestB1'], x_test['ClosestB2'], x_test['ClosestB3'], x_test['ClosestB4'], 
            x_test['ClosestB1Label_GST'], x_test['ClosestB1Label_address'], x_test['ClosestB1Label_date'],
       x_test['ClosestB1Label_description'], x_test['ClosestB1Label_email'],
       x_test['ClosestB1Label_name'], x_test['ClosestB1Label_none'], x_test['ClosestB1Label_ph_no'],
       x_test['ClosestB1Label_sac'],x_test['ClosestB1Label_tax_percent'],
       x_test['ClosestB1Label_tot_in_words'], x_test['ClosestB1Label_value'],
            
       x_test['ClosestB2Label_GST'], x_test['ClosestB2Label_address'], x_test['ClosestB2Label_date'],
       x_test['ClosestB2Label_description'], x_test['ClosestB2Label_email'],
       x_test['ClosestB2Label_name'], x_test['ClosestB2Label_none'], x_test['ClosestB2Label_ph_no'],
       x_test['ClosestB2Label_sac'],x_test['ClosestB2Label_tax_percent'],
       x_test['ClosestB2Label_tot_in_words'], x_test['ClosestB2Label_value'],    
            
       x_test['ClosestB3Label_GST'], x_test['ClosestB3Label_address'], x_test['ClosestB3Label_date'],
       x_test['ClosestB3Label_description'], x_test['ClosestB3Label_email'],
       x_test['ClosestB3Label_name'], x_test['ClosestB3Label_none'], x_test['ClosestB3Label_ph_no'],
       x_test['ClosestB3Label_sac'],x_test['ClosestB3Label_tax_percent'],
       x_test['ClosestB3Label_tot_in_words'], x_test['ClosestB3Label_value'],
            
      x_test['ClosestB4Label_GST'], x_test['ClosestB4Label_address'], x_test['ClosestB4Label_date'],
       x_test['ClosestB4Label_description'], x_test['ClosestB4Label_email'],
       x_test['ClosestB4Label_name'], x_test['ClosestB4Label_none'], x_test['ClosestB4Label_ph_no'],
       x_test['ClosestB4Label_sac'],x_test['ClosestB4Label_tax_percent'],
       x_test['ClosestB4Label_tot_in_words'], x_test['ClosestB4Label_value'], 
            
            
            
            
            
       x_test['MidPointX'],
       x_test['MidPointY'], 
       
            
       x_test['ClosestBC1X'],  x_test['ClosestBC1Y'],  x_test['ClosestBC2X'], x_test['ClosestBC2Y'], x_test['ClosestBC3X'], x_test['ClosestBC3Y'], x_test['ClosestBC4X'], x_test['ClosestBC4Y']],
                         
                         y_test),
         
        epochs=20,
         class_weight=class_weights,
         batch_size=16,
         )

Train on 37078 samples, validate on 6544 samples
Epoch 1/20
37078/37078 [==============================] - 22529s 608ms/sample - loss: 0.3799 - acc: 0.8848 - val_loss: 0.2586 - val_acc: 0.9176
Epoch 2/20
27968/37078 [=====================>........] - ETA: 1:20:25 - loss: 0.1984 - acc: 0.9379